In [1]:
from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
casas=["1xBet","10x10bet","Alphabet","bet-at-home","bet365","BetInAsia","GGBET","Lasbet","Marsbet","Pinnacle","Unibet","VOBET","Vulkan Bet","William Hill"]
rankings_fifa_2022=[
    ('Netherlands', 8), 
    ('Senegal', 18), 
    ('Ecuador', 44), 
    ('Qatar', 50),
    ('England', 5), 
    ('U.S.', 16), 
    ('Wales', 19), 
    ('Iran', 20),
    ('Argentina', 3),
    ('Mexico', 13),
    ('Poland', 26),
    ('Saudi Arabia', 51),
    ('France', 4),
    ('Denmark', 10),
    ('Tunisia', 30),
    ('Australia', 38),
    ('Spain', 7),
    ('Germany', 11),
    ('Japan', 24),
    ('Costa Rica', 31),
    ('Belgium', 2),
    ('Croatia', 12),
    ('Morocco', 22),
    ('Canada', 41),
    ('Brazil', 1),
    ('Switzerland', 15),
    ('Serbia', 21),
    ('Cameroon', 43),
    ('Portugal', 9),
    ('Uruguay', 14),
    ('South Korea', 28),
    ('Ghana', 61)]
rankings_fifa_2018=[
    ("Germany", 1),
    ("Brazil", 2),
    ("Belgium", 3),
    ("Portugal", 4),
    ("Argentina", 5),
    ("Switzerland", 6),
    ("France", 7),
    ("Poland", 8),
    ("Spain", 10),
    ("Peru", 11),
    ("Denmark", 12),
    ("England", 13),
    ("Mexico", 14),
    ("Colombia", 15),
    ("Uruguay", 17),
    ("Croatia", 18),
    ("Tunisia", 21),
    ("Iceland", 22),
    ("Costa Rica", 23),
    ("Sweden", 24),
    ("Senegal", 28),
    ("Serbia", 34),
    ("Australia", 36),
    ("Iran", 37),
    ("Nigeria", 48),
    ("Morocco", 42),
    ("Japan", 61),
    ("South Korea", 57),
    ("Saudi Arabia", 67),
    ("Russia", 70),
    ("Panama", 55)
]

In [229]:
rankings_fifa_2022.sort(key=lambda x:x[1])
superiores_2018=[]
superiores_2022=[]
for i in range(8): 
    superiores_2022.append(rankings_fifa_2022[i][0])
    superiores_2018.append(rankings_fifa_2018[i][0])
    print(rankings_fifa_2022[i],rankings_fifa_2018[i])

('Brazil', 1) ('Germany', 1)
('Belgium', 2) ('Brazil', 2)
('Argentina', 3) ('Belgium', 3)
('France', 4) ('Portugal', 4)
('England', 5) ('Argentina', 5)
('Spain', 7) ('Switzerland', 6)
('Netherlands', 8) ('France', 7)
('Portugal', 9) ('Poland', 8)


In [319]:

def computar_antigua(cuotas,consenso):
    
    temp=[]
    a_priori=1

    for i in range(3): 
        temp.append([1/cuotas[i][0],1/cuotas[i][1],cuotas[i][2]])
        a_priori*=cuotas[i][0]
        
    prob_dos_primeros=1
    ganaron_primero=0
    
    if(temp[-1][2][0]>temp[-1][2][1]): #Gana
        prob_dos_primeros=temp[-2][1]*temp[-1][1] #P(G2|G1)P(G1)
        ganaron_primero=1
    else: #Pierde o empata
        prob_dos_primeros=temp[-2][0]-temp[-2][1]*(1-temp[-1][1]) #P(G2)-P(G2|~G1)P(~G1)
    
    #print(prob_dos_primeros)
    
    prob_final=1
    
    if(temp[-2][2][0]>temp[-2][2][1] and ganaron_primero==1): #Ganaron los dos partidos previos
        ganaron_primero=1
        prob_final=temp[-3][1]*prob_dos_primeros
    else:
        ganaron_primero=0
        prob_final=temp[-3][0]-temp[-3][1]*(1-prob_dos_primeros)

    return a_priori,1/prob_final,ganaron_primero

def computar_true_p(cuotas,consenso):
    
    temp=[]
    consenso_temp=[]
    a_priori=1

    for i in range(3): 
        temp.append([1/cuotas[i][0],1/cuotas[i][1],cuotas[i][2]])
        consenso_temp.append([1/consenso[i][0],1/consenso[i][1]])
        a_priori*=cuotas[i][0]
    
    print("%"*50)
    print(temp)
    print(consenso_temp)
    print("%"*50)
    
    prob_dos_primeros=1
    ganaron_primero=0
    
    if(temp[-1][2][0]>temp[-1][2][1]): #Gana
        prob_dos_primeros=consenso_temp[-2][1]*consenso_temp[-1][1] #P(G2|G1)P(G1)
        ganaron_primero=1
    else: #Pierde o empata
        prob_dos_primeros=consenso_temp[-2][0]-consenso_temp[-2][1]*(1-consenso_temp[-1][1]) #P(G2)-P(G2|~G1)P(~G1)
    
    print("IMPRIMIR",prob_dos_primeros,consenso_temp[-3][0])
    
    if(prob_dos_primeros<0):
        print("NEGATIVO")
        return a_priori,10000

    prob_final=1
    
    if(temp[-2][2][0]>temp[-2][2][1] and ganaron_primero==1): #Ganaron los dos partidos previos
        prob_final=consenso_temp[-3][1]*prob_dos_primeros
    else:
        prob_final=consenso_temp[-3][0]-consenso_temp[-3][1]*(1-prob_dos_primeros)

    if(prob_final<0):
        print("NEGATIVO")    
        return a_priori,10000
    
    return a_priori,1/prob_final


def explotar(casa,paises_rank,cuotas_pais,dic_infra,dic_sobre,cuotas_consenso):
    
    equipos_infra=[]
    equipos_sobre=[]
    negativos=[]
    
    #print(cuotas_pais[casa])
    
    for i in cuotas_pais:
        print("*"*50)
        print(i)
        print(cuotas_pais[i][casa])
        
        a_priori,a_posteriori=computar_true_p(cuotas_pais[i][casa],cuotas_consenso[i])

        print("Priori vs posteriori",a_priori,a_posteriori)
        print("*"*50)

        #if(a_posteriori<0): negativos.append(i)
        #else:
        
        if(a_posteriori<a_priori): equipos_infra.append((i,a_priori,a_posteriori))
        else: equipos_sobre.append((i,a_priori,a_posteriori))
        
        
    #Sortear por profit o loss
    monto=1
    dinero=0

    for i in equipos_infra:
        
        dic_infra[i[0][0:3]]+=1
        prob_real=1/i[2]
        diff=prob_real*(i[1]-1)*monto-(1-prob_real)*monto
         
        dinero=diff+dinero
        #if(i[0] not in paises_rank): dinero=diff+dinero
    
    print("DINERO +",dinero)
    
    sort_perder=[]
    
    for i in equipos_sobre:

        prob_real=1/i[2]
        diff=prob_real*(i[1]-1)*monto-(1-prob_real)*monto
        sort_perder.append((i[0],diff))
        dinero=diff+dinero

        #if(i[0] not in paises_rank): dinero=diff+dinero
    
    print("Negativos")
    for i in negativos:
        print(i)
    
    print("Infravalorados")
    
    for i in equipos_infra:
        print(i)
    
    print()
    print("Sobrevalorados")
    
    for i in equipos_sobre:
        print(i)
    
    #sort_perder.sort(key=lambda x:x[1])
    #for i in cuotas_pais["Costa Rica"]:
        #print(cuotas_pais["Costa Rica"][i])
    print("Dinero despues",dinero)
    print("-"*50)
    
    return dinero

In [320]:
def analisis(year,paises_rank):
    
    df_general=pd.read_csv(f"./data_{year}/todo.csv")
    df_casas={}
    df_prob_reales=defaultdict(lambda:[])
    dic_infra=defaultdict(lambda:0)
    dic_sobre=defaultdict(lambda:0)
    cuotas_pais=defaultdict(lambda:defaultdict(lambda:[]))
    casas_validas=defaultdict(lambda:1)
    sumar=0
    
    for i in casas:
        try: df_casas[i]=pd.read_csv(f"./data_{year}/{i}.csv")
        except: casas_validas[i]=0

    for index,rows in df_general.iterrows():
    
        if(rows["tipo"]=="Groups"):

            pais_a=rows["pais_a"]
            pais_b=rows["pais_b"]
            encuentro=rows["encuentro"]
            res_a=rows["res_a"]
            res_b=rows["res_b"]
            
            arr_consenso=[0,0,0,0,0,0]
            num=0
            
            for i in casas:

                try:
                
                    odds_casa=df_casas[i][df_casas[i]["encuentro"]==encuentro]
                    casas_validas[i]=1*casas_validas[i]
                    
                    #suma_prob_inicial=1/odds_casa["inicial_0"].values[0]+1/odds_casa["inicial_x"].values[0]+1/odds_casa["inicial_1"].values[0]
                    #suma_prob_final=1/odds_casa["final_1"].values[0]+1/odds_casa["final_x"].values[0]+1/odds_casa["final_2"].values[0]
                    
                    cuotas_pais[pais_a][i].append([odds_casa["inicial_0"].values[0],odds_casa["final_1"].values[0],(res_a,res_b)])
                    cuotas_pais[pais_b][i].append([odds_casa["inicial_1"].values[0],odds_casa["final_2"].values[0],(res_b,res_a)])
                
                    arr_consenso[0]+=odds_casa["inicial_0"].values[0]
                    arr_consenso[1]+=odds_casa["inicial_x"].values[0]
                    arr_consenso[2]+=odds_casa["inicial_1"].values[0]
                    arr_consenso[3]+=odds_casa["final_1"].values[0]
                    arr_consenso[4]+=odds_casa["final_x"].values[0]
                    arr_consenso[5]+=odds_casa["final_2"].values[0]
                    num+=1
                    #print(1/(odds_casa["inicial_0"].values[0]*suma_prob_inicial)+1/(odds_casa["inicial_x"].values[0]*suma_prob_inicial)+1/(odds_casa["inicial_1"].values[0]*suma_prob_inicial))
                    #print(1/(odds_casa["final_1"].values[0]*suma_prob_final)+1/(odds_casa["final_x"].values[0]*suma_prob_final)+1/(odds_casa["final_2"].values[0]*suma_prob_final))
                
                                                                                   
                except Exception as e:
                    print(f"La casa {i}, no saco cuota para el partido {encuentro}")
                    casas_validas[i]=0*casas_validas[i]
            
            #print("$"*50)
            #print(avg_inicial_0,avg_inicial_x,avg_inicial_1,avg_final_0,avg_final_x,avg_final_1)
            #print("$"*50)

            suma_prob_final=0
            suma_prob_inicial=0
            
            for i in range(6): arr_consenso[i]/=num
                
            if(pais_a=="Panama" or pais_b=="Panama"):
                print("@"*50)
                print(arr_consenso)
                print("@"*50)
             
            for i in [0,2,3,5]: 
                if(max(arr_consenso)>20): arr_consenso[i]=(1/arr_consenso[i])-0.01583
                else: arr_consenso[i]=(1/arr_consenso[i])-0.05
            for i in [1,4]: 
                if(max(arr_consenso)>20):  arr_consenso[i]=(1/arr_consenso[i])-0.018
                else: arr_consenso[i]=(1/arr_consenso[i])-0.057
            
            for i in range(3): 
                suma_prob_inicial+= arr_consenso[i]
                arr_consenso[i]=1/arr_consenso[i]

            for i in range(3,6): 
                suma_prob_final+= arr_consenso[i]
                arr_consenso[i]=1/arr_consenso[i]
                
            for i in range(3): arr_consenso[i]=arr_consenso[i]*suma_prob_inicial
            for i in range(3,6): arr_consenso[i]=arr_consenso[i]*suma_prob_final
            
            for i in range(6):
                if(arr_consenso[i]<0): 
                    print(encuentro)
                    print(arr_consenso)
                    print("ERROR SUPREMO")
            df_prob_reales[pais_a].append([arr_consenso[0],arr_consenso[3]])
            df_prob_reales[pais_b].append([arr_consenso[2],arr_consenso[5]])
        
    avg=0
    validas=0
    
    for i in casas_validas:
        if(casas_validas[i]==1 and i=="bet365"):
            validas+=1
            print("-"*50)
            print(i)   
            avg+=explotar(i,paises_rank,cuotas_pais,dic_infra,dic_sobre,df_prob_reales)
            print("-"*50)
            break

    print(avg/len(casas_validas))

In [321]:
analisis(2018,superiores_2018)

La casa 10x10bet, no saco cuota para el partido england-belgium
La casa Alphabet, no saco cuota para el partido england-belgium
La casa BetInAsia, no saco cuota para el partido england-belgium
La casa GGBET, no saco cuota para el partido england-belgium
La casa Lasbet, no saco cuota para el partido england-belgium
La casa VOBET, no saco cuota para el partido england-belgium
La casa 10x10bet, no saco cuota para el partido panama-tunisia
La casa Alphabet, no saco cuota para el partido panama-tunisia
La casa BetInAsia, no saco cuota para el partido panama-tunisia
La casa GGBET, no saco cuota para el partido panama-tunisia
La casa Lasbet, no saco cuota para el partido panama-tunisia
La casa VOBET, no saco cuota para el partido panama-tunisia
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
[3.4549999999999996, 3.2525000000000004, 2.2275, 4.32875, 3.765, 1.8512499999999998]
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
La casa 10x10bet, no saco cuota para el partido japan-poland
La c

In [316]:
#[[1.66, 1.4, (2, 0)], [1.57, 1.55, (2, 0)], [1.22, 1.11, (1, 2)]]

def computar_true_p(cuotas):
    
    temp=[]
    a_priori=1

    for i in range(3): 
        temp.append([round(1/cuotas[i][0],3),round(1/cuotas[i][1],3),cuotas[i][2]])
        a_priori*=cuotas[i][0]
    
    print(temp)
    print()
    
    prob_dos_primeros=1
    ganaron_primero=0
    
    if(temp[-1][2][0]>temp[-1][2][1]): #Gana
        prob_dos_primeros=temp[-2][1]*temp[-1][1] #P(G2|G1)P(G1)
        ganaron_primero=1
    else: #Pierde o empata
        print(1-temp[-1][1])
        print(temp[-2][1])
        prob_dos_primeros=temp[-2][0]-temp[-2][1]*(1-temp[-1][1]) #P(G2)-P(G2|~G1)P(~G1)
    
    print()
    print("Prob dos primeros",prob_dos_primeros)
    print()
    
    prob_final=1
    
    if(temp[-2][2][0]>temp[-2][2][1] and ganaron_primero==1): #Ganaron los dos partidos previos
        prob_final=temp[-3][1]*prob_dos_primeros
    else:
        prob_final=temp[-3][0]-temp[-3][1]*(1-prob_dos_primeros)

    print()
    print("Prob final",prob_final)
    print()
    
        
    return a_priori,1/prob_final

cuotas=[[2.908823529411764, 2.6834415584415585, (2, 3)], [2.0875000000000004, 1.7873684210526315, (3, 3)], [7.416666666666667, 7.477739286630742, (0, 2)]]

print(sum(1/i for i in nuevo))
print()
computar_true_p(cuotas)

0.9999999999999999

[[0.344, 0.373, (2, 3)], [0.479, 0.559, (3, 3)], [0.135, 0.134, (0, 2)]]

0.866
0.559

Prob dos primeros -0.005094000000000043


Prob final -0.03090006200000006



(45.035254289215686, -32.36239461267094)

In [144]:
#Trabajemos con las probabilidades reales de todo
#Saquemos las casas que tenga sigma>1
#Usemos consenso para todo
#Y comparemos
1/9.169542976123081+1/5.4776692026204445+1/1.4116639073976707

0.9999999999999999

In [289]:
def computar_true_p(cuotas,consenso):
    
    temp=[]
    consenso_temp=[]
    a_priori=1

    for i in range(3): 
        temp.append([1/cuotas[i][0],1/cuotas[i][1],cuotas[i][2]])
        consenso_temp.append([1/consenso[i][0],1/consenso[i][1]])
        a_priori*=cuotas[i][0]
    
    print("%"*50)
    print(temp)
    print(consenso_temp)
    print("%"*50)
    
    prob_dos_primeros=1
    ganaron_primero=0
    
    if(temp[-1][2][0]>temp[-1][2][1]): #Gana
        prob_dos_primeros=consenso_temp[-2][1]*consenso_temp[-1][1] #P(G2|G1)P(G1)
        ganaron_primero=1
    else: #Pierde o empata
        print("Pierde primero")
        prob_dos_primeros=consenso_temp[-2][0]-consenso_temp[-2][1]*(1-consenso_temp[-1][1]) #P(G2)-P(G2|~G1)P(~G1)
    
    print("IMPRIMIR",1/prob_dos_primeros,consenso_temp[-3][0])
    
    if(prob_dos_primeros<0):
        print("NEGATIVO")
        return a_priori,10000

    prob_final=1
    
    if(temp[-2][2][0]>temp[-2][2][1] and ganaron_primero==1): #Ganaron los dos partidos previos
        prob_final=consenso_temp[-3][1]*prob_dos_primeros
    else:
        prob_final=consenso_temp[-3][0]-consenso_temp[-3][1]*(1-prob_dos_primeros)

    if(prob_final<0):
        print("NEGATIVO")    
        return a_priori,10000
    
    return a_priori,1/prob_final

In [291]:
casa=[[3.29, 4.33, (1, 2)], [12.0, 19.0, (1, 6)], [13.0, 19.0, (0, 3)]]
#[[0.303951367781155, 0.23094688221709006, (1, 2)], [0.08333333333333333, 0.05263157894736842, (1, 6)], [0.07692307692307693, 0.05263157894736842, (0, 3)]]
reales=[[0.2693843562606509, 0.2057455493699414], [0.023614025694128238, 0.0079981572016235], [0.015256485219444643, 0.005405302756638527]]
for i in range(3):
    for j in range(2):
        #print(reales[i][j])
        reales[i][j]=1/reales[i][j]
print()
print(casa)
print()
print(reales)
print()
computar_true_p(casa,reales)


[[3.29, 4.33, (1, 2)], [12.0, 19.0, (1, 6)], [13.0, 19.0, (0, 3)]]

[[3.712168048215911, 4.860372450642648], [42.347713725434616, 125.02880035878962], [65.54589642478619, 185.00351322076256]]

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
[[0.303951367781155, 0.23094688221709006, (1, 2)], [0.08333333333333333, 0.05263157894736842, (1, 6)], [0.07692307692307693, 0.05263157894736842, (0, 3)]]
[[0.2693843562606509, 0.2057455493699414], [0.023614025694128238, 0.0079981572016235], [0.015256485219444643, 0.005405302756638527]]
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Pierde primero
IMPRIMIR 63.86062667060915 0.2693843562606509


(513.24, 14.956492188120773)

In [312]:
arr=[[3.712168048215911, 4.860372450642648], [42.347713725434616, 125.02880035878962], [65.54589642478619, 185.00351322076256]]

print()
print(arr)
print()

for i in range(3):
    for j in range(2): arr[i][j]=1/arr[i][j]



ganar_primer_partido=arr[-1][1]
perder_primer_partido=1-ganar_primer_partido
ganar_segundo_partido_dado_perdi_primero=arr[-2][1]
ganar_segundo_partido=arr[-2][0]
print()
print("Ganar_primer_partido",ganar_primer_partido,1/ganar_primer_partido)
print("Perder_primer_partido",perder_primer_partido,1/perder_primer_partido)
print("Ganar_segundo_partido_dado_perdi_primero",ganar_segundo_partido_dado_perdi_primero,1/ganar_segundo_partido_dado_perdi_primero)
print("Ganar_segundo_partido",ganar_segundo_partido,1/ganar_segundo_partido)
print()
ganar_2_partidos=ganar_segundo_partido-perder_primer_partido*ganar_segundo_partido_dado_perdi_primero
print()
print(1/(perder_primer_partido*ganar_segundo_partido_dado_perdi_primero))
print(1/ganar_2_partidos)

#Esto no puede estar de ninguna manera bien!


[[3.712168048215911, 4.860372450642648], [42.347713725434616, 125.02880035878962], [65.54589642478619, 185.00351322076256]]


Ganar_primer_partido 0.005405302756638527 185.00351322076256
Perder_primer_partido 0.9945946972433615 1.0054346788411601
Ganar_segundo_partido_dado_perdi_primero 0.0079981572016235 125.02880035878962
Ganar_segundo_partido 0.023614025694128238 42.347713725434616


125.70829173463517
63.86062667060915


In [322]:
arr=[[3.29, 4.33, (1, 2)], [12.0, 19.0, (1, 6)], [13.0, 19.0, (0, 3)]]
#arr=[[3.712168048215911, 4.860372450642648], [42.347713725434616, 125.02880035878962], [65.54589642478619, 185.00351322076256]]

print()
print(arr)
print()

for i in range(3):
    for j in range(2): arr[i][j]=1/arr[i][j]



ganar_primer_partido=arr[-1][1]
perder_primer_partido=1-ganar_primer_partido
ganar_segundo_partido_dado_perdi_primero=arr[-2][1]
ganar_segundo_partido=arr[-2][0]
print()
print("Ganar_primer_partido",ganar_primer_partido,1/ganar_primer_partido)
print("Perder_primer_partido",perder_primer_partido,1/perder_primer_partido)
print("Ganar_segundo_partido_dado_perdi_primero",ganar_segundo_partido_dado_perdi_primero,1/ganar_segundo_partido_dado_perdi_primero)
print("Ganar_segundo_partido",ganar_segundo_partido,1/ganar_segundo_partido)
print()
ganar_2_partidos=ganar_segundo_partido-perder_primer_partido*ganar_segundo_partido_dado_perdi_primero
print()
print(1/(perder_primer_partido*ganar_segundo_partido_dado_perdi_primero))
print(1/ganar_2_partidos)

        #prob_final=consenso_temp[-3][0]-consenso_temp[-3][1]*(1-prob_dos_primeros)

ganar_3_partidos=arr[-3][0]-arr[-3][1]*(1-ganar_2_partidos)
print(1/ganar_3_partidos)
#Esto no puede estar de ninguna manera bien!


[[3.29, 4.33, (1, 2)], [12.0, 19.0, (1, 6)], [13.0, 19.0, (0, 3)]]


Ganar_primer_partido 0.05263157894736842 19.0
Perder_primer_partido 0.9473684210526316 1.0555555555555556
Ganar_segundo_partido_dado_perdi_primero 0.05263157894736842 19.0
Ganar_segundo_partido 0.08333333333333333 12.0


20.055555555555554
29.875862068965525
12.386247478589338


In [329]:
#Que quiere decir que la cuota BAJE
#1 Esta mal asumir que se puede entre a1 y a2 como un mismo evento
#2 El modelo de las casas de apuestas es muy malo y hay muchas diferencias en el mismo pricing

#[[2.2, 2.5, (2, 0)], [9.0, 6.5, (0, 1)], [3.39, 3.75, (0, 1)]]

arr=[[3.29, 4.33, (1, 2)], [13.0, 19.0, (0, 3)]]
arr=[[4.33, 4.33, (1, 3)], [5.0, 9.0, (0, 1)]]

#PERU
arr=[[9.0, 6.5, (0, 1)], [3.39, 3.75, (0, 1)]]
#
primer_partido=[3.39,3.2,2.2]
segundo_partidp=[1.39,4.2,9]
for i in range(2):
    for j in range(2): arr[i][j]=1/arr[i][j]


ganar_primer_partido=arr[-1][1]
perder_primer_partido=1-ganar_primer_partido
ganar_segundo_partido_dado_perdi_primero=arr[-2][1]
ganar_segundo_partido=arr[-2][0]

ganar_2_partidos=ganar_segundo_partido-perder_primer_partido*ganar_segundo_partido_dado_perdi_primero

dado_gana_primero=

print(1/ganar_2_partidos)


-584.9999999999926


In [357]:
def normalize_decimal_odds(odds):
    # Convert decimal odds to probabilities
    probabilities = 1 / np.array(odds)
    
    total_probability = np.sum(probabilities)
    normalized_probabilities = probabilities / total_probability
    
    # Convert the normalized probabilities back to decimal odds
    
    print(1 / normalized_probabilities)
    
    #normalized_odds = 1 / normalized_probabilities
    
    return normalized_probabilities


primer_partido=[3.39,3.2,2.2]
segundo_partido_post=[1.39,4.2,9]
segundo_partido_pre=[1.61,3.9,6.5]

suma_primer=0
suma_segundo_pre=0
suma_segundo_post=0

primer_partido=normalize_decimal_odds(primer_partido)
segundo_partido_post=normalize_decimal_odds(segundo_partido_post)
segundo_partido_pre=normalize_decimal_odds(segundo_partido_pre)

#peru
gana_primero=primer_partido[0]
gana_segundo=segundo_partido_pre[-1]
gana_segundo_dado_pierde_primero=segundo_partido_post[-1]
print()
print(1/gana_primero)
print(1/gana_segundo)
print()

gana_dos=gana_segundo-gana_segundo_dado_pierde_primero*(1-gana_primero)
gana_dos_dado_primero=(gana_dos)/gana_primero
print(1/gana_dos)
print(1/gana_dos_dado_primero)

[3.60028409 3.39849826 2.33646755]
[1.48539683 4.4882494  9.61767729]
[1.66051282 4.02236025 6.70393375]

3.600284090909091
6.703933747412009

13.500620990339772
3.749876579025961


In [ ]:
#Con las cuotas de la misma casa de apuesta (en teoria del mismo manifold)
#obtengo valores negativos
#el consenso no funciona pq estas mezclando distintos espacios de probabilidad?